In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import time

In [2]:
data = pd.DataFrame(columns=['year', 'speaker', 'speech', 'n_words'])
data

,year,speaker,speech,n_words


In [3]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1392&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page1_url = base_url + first_part + '1' + second_part
page2_url = base_url + first_part + '2' + second_part
page3_url = base_url + first_part + '3' + second_part

In [4]:
all_names = ['امامی','کاشانی','اکبری','ابوترابی','صدیقی', 'خاتمی',
             'موحدی', 'جنتی', 'خامنه', 'هاشمی', 'رفسنجانی']

In [5]:
def get_page_links(year_url):
    request = requests.get(year_url)
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    pages_to_visit = set()
    for tag in soup.find_all("a"):
        found = False
        for w in all_names + ['جمعه', 'تهران']:
            if w in str(tag) and not found:
                found = True
            elif w in str(tag) and found:
                pages_to_visit.add(base_url + tag.get('href'))
                break
        
    return list(pages_to_visit)

In [6]:
pages = get_page_links(page1_url)
pages

['https://www.isna.ir/news/91111307629/صدیقی-یال-و-کوپال-آمریکا-ریخته-است-باید-مشکل-گرانی-و-بیکاری',
 'https://www.isna.ir/news/91122315421/نماز-جمعه-این-هفته-تهران-به-امامت-جنتی-اقامه-می-شود',
 'https://www.isna.ir/news/91120402376/امامی-کاشانی-شورای-امنیت-و-سازمان-ملل-مرده-اند-حفظ-نظام-خواسته',
 'https://www.isna.ir/news/91111811479/آیت-الله-موحدی-کرمانی-خطیب-نماز-جمعه-این-هفته-تهران',
 'https://www.isna.ir/news/91111106817/نماز-جمعه-این-هفته-تهران-به-امامت-صدیقی-در-مصلی-برگزار-می-شود',
 'https://www.isna.ir/news/91120906295/کاظم-صدیقی-خطیب-نماز-جمعه-این-هفته-تهران',
 'https://www.isna.ir/news/91110201361/عسگراولادی-خطبای-نمازجمعه-با-رأفت-برخورد-کنند-خاتمی-نامه-شما',
 'https://www.isna.ir/news/91110805082/نمازجمعه-این-هفته-تهران-در-مصلی-برگزار-می-شود',
 'https://www.isna.ir/news/91110402927/نماز-جمعه-این-هفته-تهران-به-امامت-امامی-کاشانی-اقامه-می-شود',
 'https://www.isna.ir/news/91102917245/گزنده-است-که-بگویند-سران-فتنه-سران-فتنه-نبودند-معتقد-نیستیم',
 'https://www.isna.ir/news/911205

In [7]:
def get_name(speech):
    for name in all_names:
        if name in speech[:300]:
            return name
    return 'NA'

In [8]:
def get_text(pages, index, year):
    request = requests.get(pages[index])
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    speech = []
    name = 'NA'
    for p in soup.find_all('p'):
        p = re.sub(r'<.+?>', '', str(p))
        p = re.sub(r'\u200c', ' ', str(p))
        p = re.sub(r'،', '', str(p))
        p_lst = list(p)
        for i, char in enumerate(p_lst):
            if ord(char) > 2000 or ord(char) < 1000:
                p_lst[i] = ' '
        p = "".join(p_lst)
        words_lst = p.split()
        speech.extend(words_lst)
    n_words = len(speech)
    if n_words > 300:
        name = get_name(speech)
   
    return {'year': year, 'speaker': name, 'speech': speech, 'n_words': n_words}

In [9]:
# 1391
for page_url in [page1_url, page2_url, page3_url]:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1391')
        if row['n_words'] > 600 and row['speaker'] == 'NA':
            data.loc[len(data.index)] = row

In [10]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1393&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page1_url = base_url + first_part + '1' + second_part
page2_url = base_url + first_part + '2' + second_part

In [11]:
# 1392
for page_url in [page1_url, page2_url]:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1392')
        if row['n_words'] > 600 and row['speaker'] != 'NA':
            data.loc[len(data.index)] = row

In [12]:
data

,year,speaker,speech,n_words
0,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, بیان,...",1165
1,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, گفت, دولتمردان...",1107
2,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",696
3,1392,امامی,"[خطیب, جمعه, این, هفته, تهران, با, حمایت, از, ...",1234
4,1392,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1212
5,1392,موحدی,"[امام, جمعه, ی, موقت, تهران, با, بیان, این, که...",929
6,1392,صدیقی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1202
7,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, تاکید, کرد, که...",984
8,1392,امامی,"[خطیب, نماز, جمعه, تهران, خطاب, به, سرمداران, ...",1487
9,1392,موحدی,"[امام, جمعه, موقت, تهران, با, یادآوری, این, که...",1388


In [13]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1394&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page_urls = [base_url + first_part + str(page_n) + second_part  for page_n in range(1,11)]

In [14]:
# 1393
for page_url in page_urls:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1393')
        if row['n_words'] > 500 and row['speaker'] != 'NA':
            data.loc[len(data.index)] = row
    time.sleep(5)

In [15]:
data

,year,speaker,speech,n_words
0,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, بیان,...",1165
1,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, گفت, دولتمردان...",1107
2,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",696
3,1392,امامی,"[خطیب, جمعه, این, هفته, تهران, با, حمایت, از, ...",1234
4,1392,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1212
5,1392,موحدی,"[امام, جمعه, ی, موقت, تهران, با, بیان, این, که...",929
6,1392,صدیقی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1202
7,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, تاکید, کرد, که...",984
8,1392,امامی,"[خطیب, نماز, جمعه, تهران, خطاب, به, سرمداران, ...",1487
9,1392,موحدی,"[امام, جمعه, موقت, تهران, با, یادآوری, این, که...",1388


In [16]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1395&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page_urls = [base_url + first_part + str(page_n) + second_part  for page_n in range(1,12)]

In [17]:
# 1394
for page_url in page_urls:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1394')
        if row['n_words'] > 400 and row['speaker'] != 'NA':
            data.loc[len(data.index)] = row
    time.sleep(5)

In [18]:
data

,year,speaker,speech,n_words
0,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, بیان,...",1165
1,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, گفت, دولتمردان...",1107
2,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",696
3,1392,امامی,"[خطیب, جمعه, این, هفته, تهران, با, حمایت, از, ...",1234
4,1392,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1212
5,1392,موحدی,"[امام, جمعه, ی, موقت, تهران, با, بیان, این, که...",929
6,1392,صدیقی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1202
7,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, تاکید, کرد, که...",984
8,1392,امامی,"[خطیب, نماز, جمعه, تهران, خطاب, به, سرمداران, ...",1487
9,1392,موحدی,"[امام, جمعه, موقت, تهران, با, یادآوری, این, که...",1388


In [19]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1396&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page_urls = [base_url + first_part + str(page_n) + second_part  for page_n in range(1,12)]

In [20]:
# 1395
for page_url in page_urls:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1395')
        if row['n_words'] > 400 and row['speaker'] != 'NA':
            data.loc[len(data.index)] = row
    time.sleep(5)

In [21]:
data

,year,speaker,speech,n_words
0,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, بیان,...",1165
1,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, گفت, دولتمردان...",1107
2,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",696
3,1392,امامی,"[خطیب, جمعه, این, هفته, تهران, با, حمایت, از, ...",1234
4,1392,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1212
5,1392,موحدی,"[امام, جمعه, ی, موقت, تهران, با, بیان, این, که...",929
6,1392,صدیقی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1202
7,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, تاکید, کرد, که...",984
8,1392,امامی,"[خطیب, نماز, جمعه, تهران, خطاب, به, سرمداران, ...",1487
9,1392,موحدی,"[امام, جمعه, موقت, تهران, با, یادآوری, این, که...",1388


In [22]:
base_url = 'https://www.isna.ir'
first_part = '/page/archive.xhtml?mn=1&wide=0&dy=1&ms=0&pi='
second_part = '&yr=1397&kw=%D9%86%D9%85%D8%A7%D8%B2+%D8%AC%D9%85%D8%B9%D9%87+'
page_urls = [base_url + first_part + str(page_n) + second_part  for page_n in range(1,6)]

In [23]:
# 1396
for page_url in page_urls:
    pages = get_page_links(page_url)
    for index, _ in enumerate(pages):
        row =  get_text(pages, index, '1396')
        if row['n_words'] > 400 and row['speaker'] != 'NA':
            data.loc[len(data.index)] = row
    time.sleep(15)

In [24]:
data

,year,speaker,speech,n_words
0,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, بیان,...",1165
1,1392,موحدی,"[خطیب, موقت, نماز, جمعه, تهران, گفت, دولتمردان...",1107
2,1392,امامی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",696
3,1392,امامی,"[خطیب, جمعه, این, هفته, تهران, با, حمایت, از, ...",1234
4,1392,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, با, اشاره...",1212
...,...,...,...,...
69,1396,صدیقی,"[امام, جمعه, موقت, تهران, اظهار, کرد, مسئولان,...",1107
70,1396,صدیقی,"[خطیب, نماز, جمعه, این, هفته, تهران, پیروزی, ز...",865
71,1396,خاتمی,"[خطیب, نماز, جمعه, این, هفته, تهران, استعفای, ...",1216
72,1396,امامی,"[آیت, الله, امامی, کاشانی, گفت, بی, ادبی, رئیس...",642


In [25]:
data.to_csv('../data/speeches_91to96.csv', sep='\t')